In [4]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense,Conv2D,Conv2DTranspose,Flatten,Reshape,RepeatVector,LSTM,LeakyReLU,Dropout
from keras.models import Sequential
from keras.optimizers import Adam

(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

train_x = train_x.astype(np.float32) / 255.
test_x = test_x.astype(np.float32) / 255.


def make_di_m():
    panbyeol = Sequential([
        Conv2D(64,3,padding='same',activation=LeakyReLU(0.2),input_shape= (28, 28, 1)),
        Conv2D(128,3,2,padding='same',activation=LeakyReLU(0.2)),
        Conv2D(128,3,2,padding='same',activation=LeakyReLU(0.2)),
        Conv2D(256,3,2,padding='same',activation=LeakyReLU(0.2)),
        Flatten(),
        Dropout(0.4),
        Dense(10,activation='sigmoid')
    ])

def make_g_m():
    sangsung = Sequential([
        Dense(4*4*256,activation=LeakyReLU(0.2),input_dim = 100),
        Reshape((4,4,256)),
        Conv2DTranspose(128,4,2,padding='same', activation=LeakyReLU(0.2)),
        Conv2DTranspose(128,4,2,padding='same', activation=LeakyReLU(0.2)),
        Conv2DTranspose(128,4,2,padding='same', activation=LeakyReLU(0.2)),
        Conv2D(3, 3, padding = 'same', activation='tanh')
    ])

def make_gan_m(G, D):
  D.trainable = False
  m = Sequential([G,D])
  m.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
  return m

epochs = 10
batch_size = 32
c_size = 100

def g_re_img(data,bc_size):
  idx = np.random.randint(0,data.shape[0],bc_size)
  x = data[idx]
  y = np.ones((bc_size,1))
  return x,y

def g_g_img(G,c_size,bc_size):
  z = np.random.randn(bc_size,c_size)
  x = G.predict(z,verbose=0)
  y = np.zeros((bc_size,1))
  return x,y

def train(G,D,GAN,data,c_size,bc_size,ep,v=0):
  n_bc=int(data.shape[0]/bc_size)
  for i in range(1,ep+1):
    for bc in range(n_bc):
      re_x,re_y=g_re_img(data,bc_size//2)
      d_loss1 = D.train_on_batch(re_x,re_y)
      g_x,g_y=g_g_img(G,c_size,bc_size//2)
      d_loss2 = D.train_on_batch(g_x,g_y)

      z = np.random.randn(bc_size,c_size)
      gn_y = np.ones((bc_size,1))
      g_loss = GAN.train_on_batch(z,gn_y)

    if v==1:
      print(f"{i}회: 판별_loss(re):{d_loss1[0]}, 판별_loss(g):{d_loss2[0]},GAN_loss:{g_loss}")
    if i%10==0:
      plt.figure(figsize=(20,2))
      plt.suptitle(f"E:{i}")
      ck_x,ck_y=g_g_img(G,c_size,10)
      for n in range(10):
        plt.subplot(1,10,n+1)
        plt.imshow((ck_x[n]+1)/2.0,cmap='gray')
        plt.axis('off')
      plt.show()

D = make_di_m()
G = make_g_m()
GAN = make_gan_m(G,D)
train(G, D, GAN, train_x, c_size, batch_size, epochs, 1)

AttributeError: ignored

In [3]:
import cv2
import numpy as np
import sys

def c_yolov3():
    f = open('coco_names.txt', 'r')
    c_l = [i.strip() for i in f]
    m = cv2.dnn.readNet("yolov3.weights", 'yolov3.cfg')

    l_name = m.getLayerNames()
    out_ls = [l_name[i - 1] for i in m.getUnconnectedOutLayers()]

    return m,out_ls,c_l

model, out_ls, class_name = c_yolov3()
colors = np.random.uniform(0, 255, (len(class_name), 3))
img = cv2.imread("lenna.png")
if img is None:
    sys.exit("Image load failed")

def yolov3_detect(img,model, out_ls):
    oh,ow = img.shape[0], img.shape[1]
    t_img = cv2.dnn.blobFromImage(img, 1.0/256, (416, 416), (0, 0, 0), swapRB=True)
    model.setInput(t_img)
    out_3 = model.forward(out_ls)

    box, conf, id = [], [], []
    for out_put in out_3:
        for vec85 in out_put:
            sc = vec85[5:]
            c_l_id = np.argmax(sc)
            conf_d = sc[c_l_id]
            if conf_d > 0.7:
                c_x, c_y = int(vec85[0] * ow), int(vec85[1] * oh)
                w, h = int(vec85[2] * ow), int(vec85[3] * oh)
                x, y = int(c_x - w / 2), int(c_y - h / 2)
                box.append([x, y, x+w, y+h])
                conf.append(float(conf_d))
                id.append(c_l_id)
    ind = cv2.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    oj= [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return oj

res = yolov3_detect(img, model, out_ls)

for i in range(len(res)):
    sx, sy, ex, ey, conf, id = res[i]
    text = str(class_name[id]) + f'{conf:.3f}'
    cv2.rectangle(img, (sx, sy), (ex, ey), colors[id], 2)
    cv2.putText(img, text, (sx, sy + 30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
cv2.imshow("dst", img)
cv2.waitKey()
cv2.destroyAllWindows()

DisabledFunctionError: ignored